In [ ]:
#Importing required modules and classes
from imutils.perspective import four_point_transform
from imutils import contours
import imutils
import pandas as pd
import numpy as np
import glob
import pylab as plt
from PIL import Image
import pytesseract
from pytesseract import image_to_string 
import cv2
import os
#Adding tesseract to the enivornment
pytesseract.pytesseract.tesseract_cmd = '/home/deepanshu/Downloads/leptonica-1.79.0/tesseract'

In [ ]:
#Creating a empty list to store outputs(extracted license plates)
a=[]
i=1
while(i<=44):
    #Reading Images
    image = cv2.imread("final ds/img ("+str(i)+").jpg")

    #Preprocessing image for finding out the number plate on the image
    image = imutils.resize(image, height=500)
    #gray --- > blurred --- > edged --- > find contours
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #cv2.imshow("Gray", gray)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    #cv2.imshow("Blurred", blurred)
    edged = cv2.Canny(blurred, 50, 200, 255)
    #cv2.imshow("Edged", edged)
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    displayCnt = None
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            displayCnt = approx
            break
    if displayCnt is not None:
        warped = four_point_transform(gray, displayCnt.reshape(4, 2))
        output = four_point_transform(image, displayCnt.reshape(4, 2))
        cv2.imwrite("D:/proj/licplates/lic"+str(i)+".jpeg",output)
    #Output here is the numberplates
    image = output
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    filename = "{}.png".format(os.getpid())
    cv2.imwrite(filename, gray)
    #cv2.imshow("Output", gray)
    #cv2.waitKey(0)
    text = image_to_string(Image.open(filename))
    os.remove(filename)
    a.append(text)
    i=i+1    

In [ ]:
#Adding all the texts in the list to csv
data=pd.DataFrame(a)
data.columns=['number']
data.to_csv("output.csv")
data

In [ ]:
#Accpeting Input Image Currectly as check.img and processing in the same manner as above to extract license plate
image=cv2.imread("check2.jpg")
image = imutils.resize(image, height=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blurred, 50, 200, 255)
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
displayCnt = None
for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    if len(approx) == 4:
        displayCnt = approx
        break
if displayCnt is not None:
    warped = four_point_transform(gray, displayCnt.reshape(4, 2))
    output = four_point_transform(image, displayCnt.reshape(4, 2))
gray = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
filename = "{}.png".format(os.getpid())
cv2.imwrite(filename, gray)
txt = pytesseract.image_to_string(Image.open(filename))
os.remove(filename)
print(txt)

In [ ]:
#Importing dataset from memory
df = pd.read_csv("output.csv")

In [ ]:
#Checking if the new extracted license plate is a part of dataset
z=0
for i in range(0,len(df['number'])):
    if(txt == df['number'][i]):
        z=1
        y=i+1
        break
if(z==1):
    print("Resident")
    print("Parking Slot=",y)
else:
    print("Visitor")
    print(txt)